In [1]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from dotenv import load_dotenv

In [2]:
mopdel_1 = True
def model_2_init():
    input_shape=(255,255,3)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32,kernel_size=(5,5),input_shape=input_shape,padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(64,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(4,4)))
    model.add(tf.keras.layers.Conv2D(128,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(160,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))
    model.add(tf.keras.layers.Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(30,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(2,activation='softmax'))

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.Recall()])
    return model


In [3]:
model = model_2_init()

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 255, 255, 64)      51264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 128)       131200    
                                                                 
 conv2d_3 (Conv2D)           (None, 63, 63, 160)       327840    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 160)      0         
 2D)                                                    

In [5]:
dir_train = os.environ.get('TRAIN_PATH')
dir_test = os.environ.get('TEST_PATH')
dir_val = os.environ.get('VAL_PATH')

In [6]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

Found 130 files belonging to 2 classes.


In [7]:
dataset_val = image_dataset_from_directory(
    dir_val,
    labels = "inferred",
    label_mode = "categorical",
    seed=123,
    image_size=(255,255),
    batch_size=8
)

Found 318 files belonging to 2 classes.


In [8]:
dataset_test = image_dataset_from_directory(
    dir_test,
    labels = 'inferred',
    label_mode = 'categorical',
    seed=123,
    image_size=(255,255),
    batch_size=8,
)

Found 130 files belonging to 2 classes.


In [9]:
# datagen
datagen = ImageDataGenerator(
    featurewise_center = False,
    featurewise_std_normalization = False,
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    zoom_range = (0.8, 1.2),
)

In [11]:
# dataset_train = datagen.flow(dataset_train)

# Apply data augmentation and prefetch the data
# dataset_train = dataset_train.map(lambda x, y: (datagen.random_transform(x), y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [12]:
#es_recall = EarlyStoppingByRecall(monitor='val_recall', value=0.90, verbose=1)
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
model = model_2_init()
history = model.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

Epoch 1/20


2023-12-16 16:47:08.991384: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/17 [==============================] - 24s 1s/step - loss: 83.2891 - recall_1: 0.5846 - val_loss: 0.6096 - val_recall_1: 0.9057
Epoch 2/20
17/17 [==============================] - 23s 1s/step - loss: 0.6125 - recall_1: 0.7538 - val_loss: 0.3059 - val_recall_1: 0.9057
Epoch 3/20
17/17 [==============================] - 23s 1s/step - loss: 0.4513 - recall_1: 0.8231 - val_loss: 0.2265 - val_recall_1: 0.8994
Epoch 4/20
17/17 [==============================] - 23s 1s/step - loss: 0.6963 - recall_1: 0.7538 - val_loss: 0.3856 - val_recall_1: 0.9057


In [20]:
# read Datasets/Dataset_test/1 and put the path of each image in a list
def load_datasets_test_1():
    path_test = os.environ.get('TEST_PATH')
    path_test = path_test + '/1'
    list_test = []
    for filename in os.listdir(path_test):
        list_test.append(path_test + '/' + filename)
    return list_test

In [21]:
# load img, resize and reshape
def load_img(path):
    img = Image.open(path)
    img = img.resize((255,255))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = np.resize(img,(255,255,3))
    img = img.reshape((-1,255,255,3))
    return img

In [22]:
# load the images and predict
img_tests_1 = load_datasets_test_1()
predictions = []
for img in img_tests_1:
    img = load_img(img)
    predictions.append(model.predict(img))

1/1 [==============================] - 0s 41ms/step


In [23]:
# Obtener la categoria con más probabilidad
for res in predictions:
    dog_id = np.argmax(res.max())
    print(dog_id)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
